# TP SVG

### Data Structures

- List of tupples for (shape, color, coordinates)?


### Functions to implement:
- computing an SVG from a genotype;
- computing a PNG from a phenotype;
- computing a phenotype from a genotype;
- computing a fitness value from a phenotype;
- mutating a genotype;
- performing a crossover between two genotypes.

## Code a skeleton application
First, focus on the overall code structure and solution representation. 
Write a first version of your main script that loads the target image, create a “blank” approximation and compute its fitness without implementing any improvement algorithm yet. 
Test your solution representation by generating a random solution with one simple geometric shape (e.g., one square aligned with the pixel grid) to ensure the code structure works as expected

In [ ]:
# Load image
from PIL import Image
import numpy as np
from SVG_shapes import SVGShape, Rect, Ellipse, Circle

def load_img(image_path: str):
    img = Image.open(image_path)
    # Creates an array: [height, width, channels]
    grid = np.array(img)
    w = img.width
    h = img.height
    return (grid, w, h)

original_img, w, h = load_img("./test.png")


# Create blank with one simple geometric shape (fixed size = less pixel for large images)
canvas = np.zeros(original_img.shape, dtype=np.uint8) # All Black
shapes = ["Rect", "Circle", "Ellipse"] # TODO: Add new shapes


# Create data structure -> list of tuples (genotype)
gen: list[SVGShape] = []


# Add 1 random (shape, color, coordinates)
new_rect = Rect(x=10, y=10, w=100, h=50, fill=(255, 0, 0))
gen.append(new_rect)


    # Create corresponding pixel grid (phenotype)

# Calculate distance between phenotype and original image (fitness)



## Heuristiques et Metaheuristiques

### Comment bien commencer?
- Prendre les couleurs de bases de l'image:
    - par exemple cadriller l'image et prendre la couleur moyenne de ces reclangle?
    - Prendre le top 5-10 des teintes et commencer avec une shape de chaque?

- Changer la taille de l'image pour qu'il y est moins de pixels et donc probleme plus simple